# Data Gathering

This workbook describes data collection from the Vetsbenefits.net web forum into a dictionary format. This will serve a future purpose of feeding into a scaleable and flexible remote-hosted MongoDB instance. Further progress on that effort is paused at this time. 

## Standard Packages

In [1]:
import pickle

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
from IPython.core.display import display, HTML

In [4]:
import time, os

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [7]:
import regex as re

In [8]:
from time import sleep
from random import randint

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
import datetime
import seaborn as sns

In [11]:
from datetime import datetime as dt

In [12]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pd.set_option('display.max_rows', 660)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 400)
pd.options.display.float_format = '{:,.10f}'.format

In [3]:
#pickle.dump(df_discussion_links, open("df_discussion_links_2020-03-09_12PM.p", "wb"))

df_discussion_links_pickle = pickle.load(open("df_discussion_links_2020-03-09_12PM.p", "rb"))

In [7]:
#pickle.dump(df_zipped_post2, open("df_zipped_post2_2020-03-09_01AM.p", "wb"))

df_zipped_post2_pickle = pickle.load(open("df_zipped_post2_2020-03-09_01AM.p", "rb"))

In [8]:
df_zipped_post2_pickle.shape

(13955, 5)

### Capture Data from Top Level Discussion Board
...using Beautiful Soup to move through pagination

In [58]:
recent_post_time = []

In [61]:
def get_links(base_url,end):
    
    #Get base source data
    source_code = requests.get(base_url)   
    soup = BeautifulSoup(source_code.text, 'html5lib')
    
    #Get topic titles
    topic_titles=[]
    for element in soup.find_all("a", class_="topictitle shRt_t"):
        topic_titles.append(element.text)
        
    #Get last post time
    last_post_time=[]
    for index, element in enumerate(soup.find_all('span',class_="timespan")):
        if index % 2 == 0:
#    for element in soup.find_all('span',class_="timespan"):
            last_post_time.append(element['title'])   
    
#     if last_post_time[24] > recent_post_time[0]:
#         last_post_time_copy = last_post_time.copy()
#         recent_post_time.insert(0,last_post_time_copy[0])
#         ret
       
    #Get links for topics
    links_with_text = []
    for a in soup.find_all('a', class_="topictitle shRt_t", href=True): 
        if a.text: 
            links_with_text.append(a['href'])
   
    #Loop through subsequent pages
    for i in list(range(25,end,25)):
        
        #Get source code
        source_code = requests.get(base_url + '&start={}'.format(i))
        soup = BeautifulSoup(source_code.text, 'html5lib')
    
        #Get topic titles
        for element in soup.find_all("a", class_="topictitle shRt_t"):
            topic_titles.append(element.text)

        #Get last post time
        for index, element in enumerate(soup.find_all('span',class_="timespan")):
            if index % 2 == 0:
        #    for element in soup.find_all('span',class_="timespan"):
                last_post_time.append(element['title'])    
        
        #Get links for topics
        for a in soup.find_all('a', class_="topictitle shRt_t", href=True): 
            if a.text: 
                links_with_text.append(a['href'])
                
        #Pause
        sleep(randint(5,10))
   
    #Zip cols and create DF        
    post_zip = zip(topic_titles, links_with_text, last_post_time) 
    zipped_list = list(post_zip) #Convert zip to list.
    df_zipped_links = pd.DataFrame(zipped_list, columns=['post','url','last_post_time'])
    
    df_zipped_links['url'] = [x[:-42] for x in df_zipped_links['url']]
    
    df_zipped_links[['last_post_hour','last_post_date']] = df_zipped_links.last_post_time.str.split("-",expand=True) 
    df_zipped_links['last_post_date'] = pd.to_datetime(df_zipped_links['last_post_date'])
    df_zipped_links['last_post_hour'] = pd.to_datetime(df_zipped_links['last_post_hour']).dt.time
    df_zipped_links['combined_time'] = df_zipped_links.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)
    
    # Filter out anything that isn't newer than most recent time saved
    #df_zipped_links = df_zipped_links[df_zipped_links['combined_time'] >= recent_post_time[0]]
    #recent_post_time.insert(0,df_zipped_links.combined_time[0])
    
    return df_zipped_links

In [62]:
df_discussion_links_test3 = get_links('https://vetsbenefits.net/discussion/all?sr=topics&search_id=newposts&discussions=1&unlimit=1',501)

<ipython-input-61-ff064c092b42>:67: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_zipped_links['combined_time'] = df_zipped_links.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)


In [63]:
df_discussion_links_test3

,post,url,last_post_time,last_post_hour,last_post_date,combined_time
0,ChampVA Claims,https://vetsbenefits.net/champva-claims-t193856,"9:55 AM - Mar 22, 2021",09:55:00,2021-03-22,2021-03-22 09:55:00
1,Monday Moring Roll Call,https://vetsbenefits.net/monday-moring-roll-call-t193855,"9:47 AM - Mar 22, 2021",09:47:00,2021-03-22,2021-03-22 09:47:00
2,PTSD Increase,https://vetsbenefits.net/ptsd-increase-t192490,"9:41 AM - Mar 22, 2021",09:41:00,2021-03-22,2021-03-22 09:41:00
3,Old or new rating?,https://vetsbenefits.net/old-or-new-rating-t193810,"9:12 AM - Mar 22, 2021",09:12:00,2021-03-22,2021-03-22 09:12:00
4,VA Travel,https://vetsbenefits.net/va-travel-t193742,"9:10 AM - Mar 22, 2021",09:10:00,2021-03-22,2021-03-22 09:10:00
5,Received First Moderna Vaccine Shot Wednesday,https://vetsbenefits.net/received-first-moderna-vaccine-shot-wednesday-t193843,"8:49 AM - Mar 22, 2021",08:49:00,2021-03-22,2021-03-22 08:49:00
6,Makes me sad,https://vetsbenefits.net/makes-me-sad-t193845,"8:09 AM - Mar 22, 2021",08:09:00,2021-03-22,2021-03-22 08:09:00
7,supper,https://vetsbenefits.net/supper-t180378,"10:42 PM - Mar 21, 2021",22:42:00,2021-03-21,2021-03-21 22:42:00
8,Saturday Moring Roll Call,https://vetsbenefits.net/saturday-moring-roll-call-t193844,"10:35 PM - Mar 21, 2021",22:35:00,2021-03-21,2021-03-21 22:35:00
9,Sunday Morning Roll Call,https://vetsbenefits.net/sunday-morning-roll-call-t193852,"10:07 PM - Mar 21, 2021",22:07:00,2021-03-21,2021-03-21 22:07:00


In [52]:
df_discussion_links_test2.shape

(6, 6)

In [53]:
df_discussion_links_test2.head(10)

,post,url,last_post_time,last_post_hour,last_post_date,combined_time
0,supper,https://vetsbenefits.net/supper-t180378,"10:42 PM - Mar 21, 2021",22:42:00,2021-03-21,2021-03-21 22:42:00
1,Saturday Moring Roll Call,https://vetsbenefits.net/saturday-moring-roll-call-t193844,"10:35 PM - Mar 21, 2021",22:35:00,2021-03-21,2021-03-21 22:35:00
2,Sunday Morning Roll Call,https://vetsbenefits.net/sunday-morning-roll-call-t193852,"10:07 PM - Mar 21, 2021",22:07:00,2021-03-21,2021-03-21 22:07:00
3,Old or new rating?,https://vetsbenefits.net/old-or-new-rating-t193810,"9:43 PM - Mar 21, 2021",21:43:00,2021-03-21,2021-03-21 21:43:00
4,Why a C&P Exam for D2 when the VA has all the records and is supplying medication?,https://vetsbenefits.net/why-a-c-p-exam-for-d2-when-the-va-has-all-the-reco-t193410,"9:31 PM - Mar 21, 2021",21:31:00,2021-03-21,2021-03-21 21:31:00
25,Thursday Morning Roll Call,https://vetsbenefits.net/thursday-morning-roll-call-t193832,"9:57 PM - Mar 21, 2021",21:57:00,2021-03-21,2021-03-21 21:57:00


#### ...Test Time Columns

In [16]:
df_discussion_links_test_time = get_links('https://vetsbenefits.net/discussion/all?sr=topics&search_id=newposts&discussions=1&unlimit=1',51)

In [17]:
df_discussion_links_test_time['url'] = [x[:-42] for x in df_discussion_links_test_time['url']]

In [292]:
df_discussion_links[['last_post_hour','last_post_date']] = df_discussion_links.last_post_time.str.split("-",expand=True) 
df_discussion_links['last_post_date'] = pd.to_datetime(df_discussion_links['last_post_date'])
df_discussion_links['last_post_hour'] = pd.to_datetime(df_discussion_links['last_post_hour']).dt.time
df_discussion_links['combined_time'] = df_discussion_links.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)
# df_discussion_links['dow'] = df_discussion_links.post_date.dt.dayofweek
# df_discussion_links['week'] = df_discussion_links.post_date.dt.to_period('W')
# df_discussion_links['week_alt']=df_discussion_links.post_date.dt.year.astype(str) + '-w' + df_zipped_post3.post_date.dt.week.astype(str)
# df_discussion_links['month'] = df_discussion_links.post_date.dt.strftime('%Y-%m')

<ipython-input-292-ed2cdfb08ff1>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_discussion_links['combined_time'] = df_discussion_links.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)


In [18]:
df_discussion_links_test_time[['last_post_hour','last_post_date']] = df_discussion_links_test_time.last_post_time.str.split("-",expand=True) 
df_discussion_links_test_time['last_post_date'] = pd.to_datetime(df_discussion_links_test_time['last_post_date'])
df_discussion_links_test_time['last_post_hour'] = pd.to_datetime(df_discussion_links_test_time['last_post_hour']).dt.time
df_discussion_links_test_time['combined_time'] = df_discussion_links_test_time.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)
# df_discussion_links_test_time['dow'] = df_discussion_links_test_time.post_date.dt.dayofweek
# df_discussion_links_test_time['week'] = df_discussion_links_test_time.post_date.dt.to_period('W')
# df_discussion_links_test_time['week_alt']=df_discussion_links_test_time.post_date.dt.year.astype(str) + '-w' + df_zipped_post3.post_date.dt.week.astype(str)
# df_discussion_links_test_time['month'] = df_discussion_links_test_time.post_date.dt.strftime('%Y-%m')

<ipython-input-18-d3bd1fff5bbf>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_discussion_links_test_time['combined_time'] = df_discussion_links_test_time.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)


In [257]:
df_discussion_links_test_time['combined_time'] = df_discussion_links_test_time.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)

<ipython-input-257-0ba1a236c888>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_discussion_links['combined_time'] = df_discussion_links.apply(lambda r : pd.datetime.combine(r['last_post_date'],r['last_post_hour']),1)


In [258]:
#df_discussion_links['combined_time'] = pd.to_datetime(df_discussion_links['last_post_date'] + ' ' + df_discussion_links['last_post_hour'])

In [19]:
df_discussion_links_test_time.shape

(75, 6)

In [34]:
df_discussion_links_test_time.url[0]

'https://vetsbenefits.net/supper-t180378'

In [47]:
recent_post_time.append('2021-03-21 21:18:00')

In [49]:
recent_post_time

['2021-03-21 21:18:00']

In [23]:
df = df_discussion_links_test_time[df_discussion_links_test_time['combined_time'] >= '2021-03-21 20:34:00']

In [24]:
df

,post,url,last_post_time,last_post_hour,last_post_date,combined_time
0,supper,https://vetsbenefits.net/supper-t180378,"9:18 PM - Mar 21, 2021",21:18:00,2021-03-21,2021-03-21 21:18:00
1,Saturday Moring Roll Call,https://vetsbenefits.net/saturday-moring-roll-call-t193844,"8:38 PM - Mar 21, 2021",20:38:00,2021-03-21,2021-03-21 20:38:00
2,Sunday Morning Roll Call,https://vetsbenefits.net/sunday-morning-roll-call-t193852,"8:34 PM - Mar 21, 2021",20:34:00,2021-03-21,2021-03-21 20:34:00
25,Thursday Morning Roll Call,https://vetsbenefits.net/thursday-morning-roll-call-t193832,"8:47 PM - Mar 21, 2021",20:47:00,2021-03-21,2021-03-21 20:47:00
26,Friday morning Roll Call,https://vetsbenefits.net/friday-morning-roll-call-t193836,"8:43 PM - Mar 21, 2021",20:43:00,2021-03-21,2021-03-21 20:43:00


In [35]:
df_discussion_links_test_time

,post,url,last_post_time,last_post_hour,last_post_date,combined_time
0,supper,https://vetsbenefits.net/supper-t180378,"9:18 PM - Mar 21, 2021",21:18:00,2021-03-21,2021-03-21 21:18:00
1,Saturday Moring Roll Call,https://vetsbenefits.net/saturday-moring-roll-call-t193844,"8:38 PM - Mar 21, 2021",20:38:00,2021-03-21,2021-03-21 20:38:00
2,Sunday Morning Roll Call,https://vetsbenefits.net/sunday-morning-roll-call-t193852,"8:34 PM - Mar 21, 2021",20:34:00,2021-03-21,2021-03-21 20:34:00
3,well i filed,https://vetsbenefits.net/well-i-filed-t178556,"8:04 PM - Mar 21, 2021",20:04:00,2021-03-21,2021-03-21 20:04:00
4,Old or new rating?,https://vetsbenefits.net/old-or-new-rating-t193810,"6:36 PM - Mar 21, 2021",18:36:00,2021-03-21,2021-03-21 18:36:00
5,Blood in urine after pelvic salvage radiation,https://vetsbenefits.net/blood-in-urine-after-pelvic-salvage-radiation-t193146,"6:31 PM - Mar 21, 2021",18:31:00,2021-03-21,2021-03-21 18:31:00
6,Why a C&P Exam for D2 when the VA has all the records and is supplying medication?,https://vetsbenefits.net/why-a-c-p-exam-for-d2-when-the-va-has-all-the-reco-t193410,"5:12 PM - Mar 21, 2021",17:12:00,2021-03-21,2021-03-21 17:12:00
7,VA Travel,https://vetsbenefits.net/va-travel-t193742,"4:37 PM - Mar 21, 2021",16:37:00,2021-03-21,2021-03-21 16:37:00
8,Received First Moderna Vaccine Shot Wednesday,https://vetsbenefits.net/received-first-moderna-vaccine-shot-wednesday-t193843,"4:24 PM - Mar 21, 2021",16:24:00,2021-03-21,2021-03-21 16:24:00
9,Marine passed from lung cancer.,https://vetsbenefits.net/marine-passed-from-lung-cancer-t193854,"1:46 PM - Mar 21, 2021",13:46:00,2021-03-21,2021-03-21 13:46:00


## Indvidual Posts

In [15]:
source_code = requests.get('https://vetsbenefits.net/female-vets-read-this-t179780-s80.html')
soup = BeautifulSoup(source_code.text, 'html5lib')   

In [27]:
source_code2 = requests.get("https://vetsbenefits.net/female-vets-read-this-t179780.html")
soup2 = BeautifulSoup(source_code2.text, 'html5lib')   

In [ ]:
# Global Values
innerdict = {}
outerdict = {}

In [288]:
#########################
###  WORKING VERSION  ###
#########################

def web_scraper(url):

    #############
    # Page Info #
    #############
    
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')


    topic_title = soup.find('h2', class_ = "topic-title").text
    
    #########################
    # Original Post Present #
    #########################
    
    original_post = soup.find('div', itemtype="https://schema.org/DiscussionForumPosting")
    if original_post:
        #for a in original_post.find('div', itemtype="https://schema.org/DiscussionForumPosting"):  
            
        #Get post ID
        original_post.find_all('i', id=True)
        post_id_raw = original_post.find_all('i',id=True)
        key = post_id_raw[0]['id']

#             post_id_raw = a.find('i',id=True)
#             post_id = post_id_raw['id']
#             key = post_id

        #Get Post Time
        post_time = soup.find('time', class_="hide",itemprop="datePublished").text

        #Get poster name
        J = str(soup.find('script', text = re.compile("POST_AUTHOR")))
        J1 = J.split('{')
        J2 = J1[1].split(',')
        post_author = J2[0].split(':')[1]    

        #Get Post Content
        post_content = soup.find('div', class_ = "content noskim").text
        
        #Is first post
        is_first_post = True

        #Get supporter badge
        if soup.find('i', class_="supporter-badge tooltip"):
            supporter_badge = True
        else:
            supporter_badge = None

        #Create nested dictionary structure
        outerdict[key] = dict(innerdict)
        outerdict[key]={'post_content':'{0}'.format(soup.find('div', class_ = "content noskim").text),
                        'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                        'topic_title':'{0}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge),
                       'is_first_post':'{0}'.format(is_first_post)}            
    
        #######################
        # Collect other posts #
        #######################
        
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}
    
        ###################
        # Loop Thru Pages #
        ###################    
        
        next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")
        if next_button:
            s = 40
            counter = 2

            while True:

                source_code = requests.get(url + '-s{}.html'.format(s))
                soup = BeautifulSoup(source_code.text, 'html5lib')

                for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

                    #Get post ID
                    post_id_raw = a.find('i',id=True)
                    post_id = post_id_raw['id']
                    key = post_id

                    #Get Post Time
                    post_time = a.find('time', class_="hide",itemprop="commentTime").text

                    #Get poster name
                    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
                    J1 = J.split('{')
                    J2 = J1[1].split(',')
                    post_author = J2[0].split(':')[1]    

                    #Get supporter badge
                    if a.find('i', class_="supporter-badge tooltip"):
                        supporter_badge = True
                    else:
                        supporter_badge = None

                    #Create nested dictionary structure
                    outerdict[key] = dict(innerdict)
                    outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                                    'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                                    'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

                next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

                if next_button:
                    s += 40
                    print("done page {}".format(counter))
                    print("next url is " + url + '-s{}.html'.format(s))
                    counter += 1
                    sleep(randint(5,10))                    
                
                else:
                    break

        else:
            return outerdict
        
    #########################################################################################
    
    ###################
    # If No Orig Post #
    ###################    
  
    else:
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip tooltipstered"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

        ###################
        # Loop Thru Pages #
        ###################    
        
        next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")
        if next_button:
            s = 40
            counter = 2

            while True:

                source_code = requests.get(url + '-s{}.html'.format(s))
                soup = BeautifulSoup(source_code.text, 'html5lib')

                for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

                    #Get post ID
                    post_id_raw = a.find('i',id=True)
                    post_id = post_id_raw['id']
                    key = post_id

                    #Get Post Time
                    post_time = a.find('time', class_="hide",itemprop="commentTime").text

                    #Get poster name
                    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
                    J1 = J.split('{')
                    J2 = J1[1].split(',')
                    post_author = J2[0].split(':')[1]    

                    #Get supporter badge
                    if a.find('i', class_="supporter-badge tooltip"):
                        supporter_badge = True
                    else:
                        supporter_badge = None

                    #Create nested dictionary structure
                    outerdict[key] = dict(innerdict)
                    outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                                    'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                                    'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

                next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

                if next_button:
                    s += 40
                    print("done page {}".format(counter))
                    print("next url is " + url + '-s{}.html'.format(s))
                    counter += 1
                    #sleep(randint(5,10))
                else:
                    break 

        else:
            return None
  
    return outerdict

In [295]:
for i in df_discussion_links.url:
    if i == 'https://vetsbenefits.net/supper-t180378':
        pass
    else:
        web_scraper(i)

done page 2
next url is https://vetsbenefits.net/voting-for-our-board-t190581-s80.html
done page 2
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s80.html
done page 3
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s120.html
done page 4
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s160.html
done page 5
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s200.html
done page 6
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s240.html
done page 7
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s280.html
done page 8
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s320.html
done page 9
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s360.html
done page 10
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s400.html
done page 11
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s440.html
done page 12
next url is https://vetsbenefits.net/mo

done page 13
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s520.html
done page 14
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s560.html
done page 15
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s600.html
done page 16
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s640.html
done page 17
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s680.html
done page 18
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s720.html
done page 19
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s760.html
done page 20
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s800.html
done page 21
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s840.html
done page 22
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s880.html
done page 23
next url is https://vetsben

In [300]:
outerdict

{'p1511002': {'post_content': 'Decision',
  'post_author': '"Battlecat"',
  'post_time': '2021-03-12T01:34',
  'topic_title': 'Army board',
  'supporter_badge': 'True',
  'is_first_post': 'True'},
 'p1511012': {'post_content': "I'm not able to see most of your documents. Maybe you could take another look at your attachments?",
  'post_author': '"PBnJ"',
  'post_time': '2021-03-12T02:13',
  'topic_title': 'Army board',
  'supporter_badge': 'None'},
 'p1511024': {'post_content': 'Brother \nIf you have Facebook, go head and look for this group and drop the question out there, there are more people out there who can answer your question. I would definitely reconsider if I was you, especially if you feel that this was an injustice outcome, hire an attorney and fight it back\n\nBest of luck \n\nhttps://www.facebook.com/groups/8861480 ... ?ref=share\n\n\nSent from my iPhone using Tapatalk',
  'post_author': '"Jobejobey"',
  'post_time': '2021-03-12T07:33',
  'topic_title': 'Army board',
  'su

____

## Appendix and Scratch

***All notes here are working versions to support functions above. It is intended only for reference and reach back upon revisiting this effort.*** 

In [25]:
for soup2

<!DOCTYPE html>
<html dir="ltr" lang="en-gb"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no" name="viewport"/><meta content="CkMysITTI9stGt0r-UBUly3_76PXrzqZ-5WaA2_3DW8" name="google-site-verification"/><meta content="#303f9f" name="theme-color"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="469687153111700" property="fb:app_id"/><meta content="website" property="og:type"/><meta content="Veterans Benefits Network" property="og:site_name"/><meta content="FEMALE VETS READ THIS" property="og:title"/><meta content="https://vetsbenefits.net/female-vets-read-this-t179780-s80.html" property="og:url"/><meta content="I would like access please! Thanks!" name="description"/><meta content="I would like access please! Thanks!" property="og:description"/><meta content="https://header.tapatalk-cdn.com/c1924176.jpeg?v=1559329679" 

In [28]:
soup2.find('h2', class_ = "topic-title").text

'FEMALE VETS READ THIS'

In [37]:
original_post = soup2.find('div', itemtype="https://schema.org/DiscussionForumPosting")
if original_post:
    print(original_post.find('div', class_ = "content noskim").text)
else: 
    print("not a first page")
    
    
print('testtest')

Although most questions regarding claims, etc can be posted in the open, we realize there may be certain discussions you may want to have in a more private setting.  We have a forum dedicated to these types of things.  The only people that can enter the forum are females, staff and a few others that are given specific access due to their knowledge, CV, etc.

If you wish to access that forum, please just post a quick reply to this thread.  

Vol
testtest


In [44]:
original_post_id = original_post.find('div', class_="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block",id=True)
#     for a in soup.find_all('a', class_="topictitle shRt_t", href=True): 
#         if a.text: 
#             links_with_text.append(a['href'])

original_post_id['id']

TypeError: 'NoneType' object is not subscriptable

In [43]:
original_post = soup2.find('div', itemtype="https://schema.org/DiscussionForumPosting")
# if
# original_post.find('div', class_ = "content noskim").text

In [56]:
original_post = soup2.find('div', itemtype="https://schema.org/DiscussionForumPosting")

In [58]:
original_post.find('time', class_="hide",itemprop="commentTime").text

AttributeError: 'NoneType' object has no attribute 'text'

In [163]:
if soup2.find('i', class_="supporter-badge tooltip"):
    print("it's true")
else:
    print("it's false/not there")

it's true


In [ ]:
soup2

In [48]:
original_post
original_post.find('div', class_ = "content noskim").text

<div class="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block" id="p_1349102" itemscope="" itemtype="https://schema.org/DiscussionForumPosting"><div class="hide" itemprop="publisher" itemscope="" itemtype="http://schema.org/Organization"><div itemprop="logo" itemscope="" itemtype="https://schema.org/ImageObject"><img src="https://groups.tapatalk-cdn.com/static/image/logo_t.png"/><meta content="https://groups.tapatalk-cdn.com/static/image/logo_t.png" itemprop="url"/><meta content="50" itemprop="width"/><meta content="50" itemprop="height"/></div><meta content="https://tapatalk.com" itemprop="url"/><meta content="Tapatalk" itemprop="name"/></div><meta content="2021-03-09 04:47:34" itemprop="dateModified"/><a class="hide" href="./viewtopic.php?f=63&amp;t=179780" itemprop="mainEntityOfPage"><h1 itemprop="name">FEMALE VETS READ THIS</h1></a><i id="p1349102"></i><i id="pfirst"></i><h1 class="hide" itemprop="headline">FEMALE VETS READ THIS<

In [47]:
original_post.find('div', class_="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block",id=True)

In [69]:
tempdic

{'topic_title': 'FEMALE VETS READ THIS',
 'original_post_content': 'Although most questions regarding claims, etc can be posted in the open, we realize there may be certain discussions you may want to have in a more private setting.\xa0 We have a forum dedicated to these types of things.\xa0 The only people that can enter the forum are females, staff and a few others that are given specific access due to their knowledge, CV, etc.\n\nIf you wish to access that forum, please just post a quick reply to this thread.\xa0 \n\nVol',
 'original_post_id': 'p_1349102',
 'original_post_time': '2017-10-16T19:19',
 'original_post_author': '"Vol"'}

In [67]:
J2[0].split(':')[1]

'"Vol"'

In [77]:
url = "https://vetsbenefits.net/covid-vaccine-distribution-t192886"

In [78]:
source_code = requests.get(url+'.html')
soup = BeautifulSoup(source_code.text, 'html5lib')

In [119]:
tempdic = {}

for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

    #Get post ID
    post_id_raw = a.find('i',id=True)
    tempdic['post_id'] = post_id_raw['id']
    
    #Get post content and add to dictionary
    tempdic['post_content'] = a.find('div', class_ = "content noskim").text
    

    
tempdic

{'post_id': 'p1499949',
 'post_content': 'Very good post Sobos, thanks for sharing.'}

In [100]:


# def web_scraper(url):

#     innerdict = {}
#     outerdict = {}

#     for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

#         #Get post ID
#         post_id_raw = a.find('i',id=True)
#         post_id = post_id_raw['id']
#         key = post_id

#         #Get Post Time
#         post_time = a.find('time', class_="hide",itemprop="commentTime").text

#         #Get poster name
#         J = str(a.find('script', text = re.compile("POST_AUTHOR")))
#         J1 = J.split('{')
#         J2 = J1[1].split(',')
#         post_author = J2[0].split(':')[1]    

#         #Is first post
#         #tempdic['topic_title'] = soup2.find('h2', class_ = "topic-title").text

#         #Get supporter badge
#         if a.find('i', class_="supporter-badge tooltip tooltipstered"):
#             supporter_badge = a.find('i', class_="supporter-badge tooltip tooltipstered")
#         else:
#             supporter_badge = None

#         #Create nested dictionary structure
#         outerdict[key] = dict(innerdict)
#         outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
#                         'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),'supporter_badge':'{0}'.format(supporter_badge)}

#     print(outerdict)

In [ ]:
#############################
# GLOBAL DATA STORES

In [107]:
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')

In [123]:
original_post = soup.find('div', itemtype="https://schema.org/DiscussionForumPosting")
original_post.find_all('i', id=True)
post_id_raw = original_post.find_all('i',id=True)
post_id_raw[0]['id']
# #         post_id = post_id_raw['id']
#         print(a)
#         print('\n \n BREAK \n\n')
#         #Get post ID
# #         post_id_raw = a.find('i',id=True)
# #         post_id = post_id_raw['id']
# #         key = post_id

'p1499175'

In [132]:
post_time = soup.find('time', class_="hide",itemprop="commentTime").text

'2020-12-18T21:43'

In [ ]:
if soup.find('i', class_="supporter-badge tooltip tooltipstered"):
    supporter_badge = a.find('i', class_="supporter-badge tooltip tooltipstered")
    print(True)
else:
    print('na')

In [162]:
print(soup.find_all('i', class_="supporter-badge tooltip tooltipstered"))

[]


In [116]:
original_post = soup.find('div', itemtype="https://schema.org/DiscussionForumPosting")
if original_post:
    for a in soup.find('div', itemtype="https://schema.org/DiscussionForumPosting"):  
        print(a)
        print('\n \n BREAK \n\n')
        #Get post ID
#         post_id_raw = a.find('i',id=True)
#         post_id = post_id_raw['id']
#         key = post_id

<div class="hide" itemprop="publisher" itemscope="" itemtype="http://schema.org/Organization"><div itemprop="logo" itemscope="" itemtype="https://schema.org/ImageObject"><img src="https://groups.tapatalk-cdn.com/static/image/logo_t.png"/><meta content="https://groups.tapatalk-cdn.com/static/image/logo_t.png" itemprop="url"/><meta content="50" itemprop="width"/><meta content="50" itemprop="height"/></div><meta content="https://tapatalk.com" itemprop="url"/><meta content="Tapatalk" itemprop="name"/></div>

 
 BREAK 


<meta content="2021-03-11 00:56:05" itemprop="dateModified"/>

 
 BREAK 


<a class="hide" href="./viewtopic.php?f=196&amp;t=192886" itemprop="mainEntityOfPage"><h1 itemprop="name">Covid vaccine distribution</h1></a>

 
 BREAK 


<i id="p1499175"></i>

 
 BREAK 


<i id="pfirst"></i>

 
 BREAK 


<h1 class="hide" itemprop="headline">Covid vaccine distribution</h1>

 
 BREAK 


<div class="post-profile-block"><dl class="postprofile" data-uid="10764548" id="profile1499175" st

In [149]:
post_time = soup.find('time', class_="hide",itemprop="commentTime").text
post_time

'2020-12-18T21:43'

In [294]:
# Global Values
innerdict = {}
outerdict = {}

In [297]:
outerdict[:5]

TypeError: unhashable type: 'slice'

In [298]:
1+1

2

In [299]:
len(outerdict)

12813

In [300]:
outerdict

{'p1511002': {'post_content': 'Decision',
  'post_author': '"Battlecat"',
  'post_time': '2021-03-12T01:34',
  'topic_title': 'Army board',
  'supporter_badge': 'True',
  'is_first_post': 'True'},
 'p1511012': {'post_content': "I'm not able to see most of your documents. Maybe you could take another look at your attachments?",
  'post_author': '"PBnJ"',
  'post_time': '2021-03-12T02:13',
  'topic_title': 'Army board',
  'supporter_badge': 'None'},
 'p1511024': {'post_content': 'Brother \nIf you have Facebook, go head and look for this group and drop the question out there, there are more people out there who can answer your question. I would definitely reconsider if I was you, especially if you feel that this was an injustice outcome, hire an attorney and fight it back\n\nBest of luck \n\nhttps://www.facebook.com/groups/8861480 ... ?ref=share\n\n\nSent from my iPhone using Tapatalk',
  'post_author': '"Jobejobey"',
  'post_time': '2021-03-12T07:33',
  'topic_title': 'Army board',
  'su

In [288]:
##########################
### THE ONE THAT WORKS ###
##########################

def web_scraper(url):

    #############
    # Page Info #
    #############
    
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')


    topic_title = soup.find('h2', class_ = "topic-title").text
    
    #########################
    # Original Post Present #
    #########################
    
    original_post = soup.find('div', itemtype="https://schema.org/DiscussionForumPosting")
    if original_post:
        #for a in original_post.find('div', itemtype="https://schema.org/DiscussionForumPosting"):  
            
        #Get post ID
        original_post.find_all('i', id=True)
        post_id_raw = original_post.find_all('i',id=True)
        key = post_id_raw[0]['id']

#             post_id_raw = a.find('i',id=True)
#             post_id = post_id_raw['id']
#             key = post_id

        #Get Post Time
        post_time = soup.find('time', class_="hide",itemprop="datePublished").text

        #Get poster name
        J = str(soup.find('script', text = re.compile("POST_AUTHOR")))
        J1 = J.split('{')
        J2 = J1[1].split(',')
        post_author = J2[0].split(':')[1]    

        #Get Post Content
        post_content = soup.find('div', class_ = "content noskim").text
        
        #Is first post
        is_first_post = True

        #Get supporter badge
        if soup.find('i', class_="supporter-badge tooltip"):
            supporter_badge = True
        else:
            supporter_badge = None

        #Create nested dictionary structure
        outerdict[key] = dict(innerdict)
        outerdict[key]={'post_content':'{0}'.format(soup.find('div', class_ = "content noskim").text),
                        'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                        'topic_title':'{0}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge),
                       'is_first_post':'{0}'.format(is_first_post)}            
    
        #######################
        # Collect other posts #
        #######################
        
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}
    
        ###################
        # Loop Thru Pages #
        ###################    
        
        next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")
        if next_button:
            s = 40
            counter = 2

            while True:

                source_code = requests.get(url + '-s{}.html'.format(s))
                soup = BeautifulSoup(source_code.text, 'html5lib')

                for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

                    #Get post ID
                    post_id_raw = a.find('i',id=True)
                    post_id = post_id_raw['id']
                    key = post_id

                    #Get Post Time
                    post_time = a.find('time', class_="hide",itemprop="commentTime").text

                    #Get poster name
                    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
                    J1 = J.split('{')
                    J2 = J1[1].split(',')
                    post_author = J2[0].split(':')[1]    

                    #Get supporter badge
                    if a.find('i', class_="supporter-badge tooltip"):
                        supporter_badge = True
                    else:
                        supporter_badge = None

                    #Create nested dictionary structure
                    outerdict[key] = dict(innerdict)
                    outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                                    'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                                    'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

                next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

                if next_button:
                    s += 40
                    print("done page {}".format(counter))
                    print("next url is " + url + '-s{}.html'.format(s))
                    counter += 1
                    sleep(randint(5,10))                    
                
                else:
                    break

        else:
            return outerdict
        
    #########################################################################################
    
    ###################
    # If No Orig Post #
    ###################    
  
    else:
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip tooltipstered"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

        ###################
        # Loop Thru Pages #
        ###################    
        
        next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")
        if next_button:
            s = 40
            counter = 2

            while True:

                source_code = requests.get(url + '-s{}.html'.format(s))
                soup = BeautifulSoup(source_code.text, 'html5lib')

                for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

                    #Get post ID
                    post_id_raw = a.find('i',id=True)
                    post_id = post_id_raw['id']
                    key = post_id

                    #Get Post Time
                    post_time = a.find('time', class_="hide",itemprop="commentTime").text

                    #Get poster name
                    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
                    J1 = J.split('{')
                    J2 = J1[1].split(',')
                    post_author = J2[0].split(':')[1]    

                    #Get supporter badge
                    if a.find('i', class_="supporter-badge tooltip"):
                        supporter_badge = True
                    else:
                        supporter_badge = None

                    #Create nested dictionary structure
                    outerdict[key] = dict(innerdict)
                    outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                                    'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                                    'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

                next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

                if next_button:
                    s += 40
                    print("done page {}".format(counter))
                    print("next url is " + url + '-s{}.html'.format(s))
                    counter += 1
                    #sleep(randint(5,10))
                else:
                    break 

        else:
            return None
  
    return outerdict

In [295]:
for i in df_discussion_links.url:
    if i == 'https://vetsbenefits.net/supper-t180378':
        pass
    else:
        web_scraper(i)

done page 2
next url is https://vetsbenefits.net/voting-for-our-board-t190581-s80.html
done page 2
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s80.html
done page 3
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s120.html
done page 4
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s160.html
done page 5
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s200.html
done page 6
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s240.html
done page 7
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s280.html
done page 8
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s320.html
done page 9
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s360.html
done page 10
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s400.html
done page 11
next url is https://vetsbenefits.net/monthly-cpi-updates-t69202-s440.html
done page 12
next url is https://vetsbenefits.net/mo

done page 13
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s520.html
done page 14
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s560.html
done page 15
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s600.html
done page 16
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s640.html
done page 17
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s680.html
done page 18
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s720.html
done page 19
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s760.html
done page 20
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s800.html
done page 21
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s840.html
done page 22
next url is https://vetsbenefits.net/post-presumptive-awards-here-t54138-s880.html
done page 23
next url is https://vetsben

In [ ]:
len

In [208]:
url = "https://vetsbenefits.net/covid-vaccine-distribution-t192886"

In [209]:
new_dict = web_scraper(url)

done page 2
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s80.html
done page 3
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s120.html
done page 4
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s160.html
done page 5
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s200.html


In [210]:
new_dict

{'p1499175': {'post_content': 'Running into snags already right out of the gate. I suppose that with an operation of that magnitude it\'s only to be expected.\n\nStill, building confidence in the proper storage, timely distribution and delivery of hundreds of millions of doses of these vaccines, let alone administration, is already leaving much to be desired.\n\nHow can things get screwed up?\xa0 Wish that I could be more optimistic but, "Let me count the ways."',
  'post_author': '"BigSweet"',
  'post_time': '2020-12-18T19:50',
  'topic_title': 'Covid vaccine distribution',
  'supporter_badge': 'True',
  'is_first_post': 'True'},
 'p1499202': {'post_content': 'What are you seeing ?',
  'post_author': '"Nick"',
  'post_time': '2020-12-18T21:43',
  'topic_title': 'Covid vaccine distribution',
  'supporter_badge': 'None'},
 'p1499231': {'post_content': "Let's count the ways. I'm curious as to what problems you are seeing. \n\nSent from my SM-G960U using Tapatalk\n\n",
  'post_author': '"

In [211]:
len(new_dict)

209

In [171]:
##########################
### THE OLD ONE.       ###
##########################

def web_scraper(url):

    #############
    # Page Info #
    #############
    
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')
    
    innerdict = {}
    outerdict = {}

    topic_title = soup.find('h2', class_ = "topic-title").text
    
    #########################
    # Original Post Present #
    #########################
    
    original_post = soup.find('div', itemtype="https://schema.org/DiscussionForumPosting")
    if original_post:
        #for a in original_post.find('div', itemtype="https://schema.org/DiscussionForumPosting"):  
            
        #Get post ID
        original_post.find_all('i', id=True)
        post_id_raw = original_post.find_all('i',id=True)
        key = post_id_raw[0]['id']

#             post_id_raw = a.find('i',id=True)
#             post_id = post_id_raw['id']
#             key = post_id

        #Get Post Time
        post_time = soup.find('time', class_="hide",itemprop="datePublished").text

        #Get poster name
        J = str(soup.find('script', text = re.compile("POST_AUTHOR")))
        J1 = J.split('{')
        J2 = J1[1].split(',')
        post_author = J2[0].split(':')[1]    

        #Get Post Content
        post_content = soup.find('div', class_ = "content noskim").text
        
        #Is first post
        is_first_post = True

        #Get supporter badge
        if soup.find('i', class_="supporter-badge tooltip"):
            supporter_badge = True
        else:
            supporter_badge = None

        #Create nested dictionary structure
        outerdict[key] = dict(innerdict)
        outerdict[key]={'post_content':'{0}'.format(soup.find('div', class_ = "content noskim").text),
                        'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                        'topic_title':'{0}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge),
                       'is_first_post':'{0}'.format(is_first_post)}            
    
        #######################
        # Collect other posts #
        #######################
        
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}
    
    
    #########################################################################################
    
    ###################
    # If No Orig Post #
    ###################    
  
    else:
        for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):

            #Get post ID
            post_id_raw = a.find('i',id=True)
            post_id = post_id_raw['id']
            key = post_id

            #Get Post Time
            post_time = a.find('time', class_="hide",itemprop="commentTime").text

            #Get poster name
            J = str(a.find('script', text = re.compile("POST_AUTHOR")))
            J1 = J.split('{')
            J2 = J1[1].split(',')
            post_author = J2[0].split(':')[1]    

            #Get supporter badge
            if a.find('i', class_="supporter-badge tooltip tooltipstered"):
                supporter_badge = True
            else:
                supporter_badge = None

            #Create nested dictionary structure
            outerdict[key] = dict(innerdict)
            outerdict[key]={'post_content':'{0}'.format(a.find('div', class_ = "content noskim").text),
                            'post_author':'{0}'.format(post_author),'post_time':'{0}'.format(post_time),
                            'topic_title':'{}'.format(topic_title),'supporter_badge':'{0}'.format(supporter_badge)}

    return outerdict

In [157]:
if soup.find('i', class_="supporter-badge tooltip tooltipstered"):
    supporter_badge = a.find('i', class_="supporter-badge tooltip tooltipstered")
    print(True)
else:
    print('na')

na


In [175]:
url = "https://vetsbenefits.net/covid-vaccine-distribution-t192886"

In [191]:
new_dict = web_scraper(url)

done page 2
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s80.html
done page 3
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s120.html
done page 4
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s160.html
done page 5
next url is https://vetsbenefits.net/covid-vaccine-distribution-t192886-s200.html


KeyboardInterrupt: 

In [ ]:
tempdic['topic_title'] = soup2.find('h2', class_ = "topic-title").text

In [ ]:
innerdict = {}
outerdict = {}
parts_1 = out.split("\n")
for line in parts_1:
     
    parts = line.split(" ")
    line_tuple = (int(parts[0]), parts[1], int(parts[2]), int(parts[3]), float(parts[4]))
    key = line_tuple[1]
    outerdict[key] = dict(innerdict)
    outerdict[key]={'Total':'{0}'.format(parts[3]),'Number':'{0}'.format(parts[0]),'Grade':'{0}'.format(parts[2]),'Weight':'{0}'.format(parts[4])}
     
   
print(outerdict)

In [ ]:
    #Get Post Time
    tempdic['original_post_time'] = a.find('time', class_="hide",itemprop="commentTime").text

    #Get poster name
    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
    J1 = J.split('{')
    J2 = J1[1].split(',')
    tempdic['original_post_author'] = J2[0].split(':')[1]

    #Get supporter badge
    if a.find('i', class_="supporter-badge tooltip tooltipstered"):
        tempdic['supporter_badge'] = a.find('i', class_="supporter-badge tooltip tooltipstered")

In [93]:
for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):
    print(a.find('div', itemtype="https://schema.org/UserComments"))
    print('\n BREAK \n BREAK \n')

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 

None

 BREAK 
 BREAK 



In [92]:
for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):
    print(a, '\n BREAK \n BREAK \n')

<div class="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block" id="p_1499202" itemscope="" itemtype="https://schema.org/UserComments"><i id="p1499202"></i><div class="post-profile-block"><dl class="postprofile" data-uid="6961625" id="profile1499202" style=""><dt class="has-profile-rank has-avatar avatar-username"><div class="avatar-username-inner responsive-hide"><div class="avatar-container"><div class="avatar-frame"><a class="avatar avatar-frame-padding" href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=6961625&amp;sid=98d186c4adfe52d0da322123d695f82c"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/avatar/455/6961625_1510236306.jpg)"></i></a></div></div><div style="padding-top:10px;"><span class="thread-user-name"><a class="username-coloured" href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=6961625&amp;sid=98d186c4adfe52d0da322123d695f82c" itemprop="name" styl

In [81]:
tempdic={}

for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):
    
    #Get original post content and add to dictionary
    tempdic['post_content'] = a.find('div', class_ = "content noskim").text

    #Get original post ID
    post_id_raw = a.find('div', class_="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block",id=True)
    tempdic['post_id'] = post_id_raw['id']

    #Get Post Time
    tempdic['original_post_time'] = a.find('time', class_="hide",itemprop="commentTime").text

    #Get poster name
    J = str(a.find('script', text = re.compile("POST_AUTHOR")))
    J1 = J.split('{')
    J2 = J1[1].split(',')
    tempdic['original_post_author'] = J2[0].split(':')[1]

    #Get supporter badge
    if a.find('i', class_="supporter-badge tooltip tooltipstered"):
        tempdic['supporter_badge'] = a.find('i', class_="supporter-badge tooltip tooltipstered")
        
tempdic   

TypeError: 'NoneType' object is not subscriptable

In [74]:
for a in soup.find_all('div', itemtype="https://schema.org/DiscussionForumPosting"):
    print(a)
    print('\n')

<div class="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block" id="p_1349102" itemscope="" itemtype="https://schema.org/DiscussionForumPosting"><div class="hide" itemprop="publisher" itemscope="" itemtype="http://schema.org/Organization"><div itemprop="logo" itemscope="" itemtype="https://schema.org/ImageObject"><img src="https://groups.tapatalk-cdn.com/static/image/logo_t.png"/><meta content="https://groups.tapatalk-cdn.com/static/image/logo_t.png" itemprop="url"/><meta content="50" itemprop="width"/><meta content="50" itemprop="height"/></div><meta content="https://tapatalk.com" itemprop="url"/><meta content="Tapatalk" itemprop="name"/></div><meta content="2021-03-09 04:47:34" itemprop="dateModified"/><a class="hide" href="./viewtopic.php?f=63&amp;t=179780" itemprop="mainEntityOfPage"><h1 itemprop="name">FEMALE VETS READ THIS</h1></a><i id="p1349102"></i><i id="pfirst"></i><h1 class="hide" itemprop="headline">FEMALE VETS READ THIS<

In [68]:
#Get soup info
source_code = requests.get(url+'.html')
soup = BeautifulSoup(source_code.text, 'html5lib')

#Setup dictionary
tempdic = {}
tempdic['topic_title'] = soup2.find('h2', class_ = "topic-title").text

original_post = soup2.find('div', itemtype="https://schema.org/DiscussionForumPosting")

#########################
# Collect Original Post #
#########################

for a in 
    if original_post:

find_all
        
        #Get original post content and add to dictionary
        tempdic['post_content'] = original_post.find('div', class_ = "content noskim").text

        #Get original post ID
        original_post_id_raw = soup2.find('div', class_="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block",id=True)
        tempdic['post_id'] = original_post_id_raw['id']

        #Get Post Time
        tempdic['original_post_time'] = soup2.find('time', class_="hide",itemprop="commentTime").text

        #Get poster name
        J = str(soup2.find('script', text = re.compile("POST_AUTHOR")))
        J1 = J.split('{')
        J2 = J1[1].split(',')
        tempdic['original_post_author'] = J2[0].split(':')[1]

        #Get supporter badge
        if soup2.find('i', class_="supporter-badge tooltip tooltipstered"):
            tempdic['supporter_badge'] = soup2.find('i', class_="supporter-badge tooltip tooltipstered")

#####################
# Get new post info #
#####################

else: 
    
    #Get post content and add to dictionary
    tempdic['post_content'] = original_post.find('div', class_ = "content noskim").text
    
    #Get original post ID
    original_post_id_raw = soup2.find('div', class_="change_color post post-bubble-padding postrow incountiously has-profile rowtargetads has-profile-block",id=True)
    tempdic['original_post_id'] = original_post_id_raw['id']

    #Get Post Time
    tempdic['original_post_time'] = soup2.find('time', class_="hide",itemprop="commentTime").text

    #Get poster name
    J = str(soup2.find('script', text = re.compile("POST_AUTHOR")))
    J1 = J.split('{')
    J2 = J1[1].split(',')
    tempdic['original_post_author'] = J2[0].split(':')[1]
    
    #Get supporter badge
    if soup2.find('i', class_="supporter-badge tooltip tooltipstered"):
        tempdic['supporter_badge'] = soup2.find('i', class_="supporter-badge tooltip tooltipstered")
    
    
# #     for a in soup.find_all('a', class_="topictitle shRt_t", href=True): 
# #         if a.text: 
# #             links_with_text.append(a['href'])

# original_post_id['id']
# else: 
#     for a in soup.find_all('div', itemtype="https://schema.org/UserComments"):
#     try:
#         #Get post IDs
#         original_post = soup.find('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
#                                           "normal-bubble post_content post-bubble-outter-padding bg2"], id=True)

#         for a in soup.find_all('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
#                                           "normal-bubble post_content post-bubble-outter-padding bg2"], id=True): 
#             post_id_container.append(a['id'])
#             original_post_id_container.append(original_post['id'])

#         #Get post text
#         for element in soup.find_all("div", class_="content noskim"):
#             post_text.append(element.text)

#         #Get Post Time
#         for a in soup.find_all('span', class_="timespan", title=True): 
#             if a.text: 
#                 post_time.append(a['title'])

#         #Get poster name
#         for element in soup.find_all('script', text = re.compile("POST_AUTHOR")):

#             J = str(element)
#             J1 = J.split('{')
#             J2 = J1[1].split(',')
#             post_author.append(J2[0].split(':'))

In [ ]:
<div id="p_1510189" itemscope="" itemtype="https://schema.org/DiscussionForumPosting" class="change_color post post-bubble-padding postrow incountiously has-profile  rowtargetads has-profile-block   runned_js"><div class="hide" itemscope="" itemprop="publisher" itemtype="http://schema.org/Organization"><div itemprop="logo" itemscope="" itemtype="https://schema.org/ImageObject"><img src="https://groups.tapatalk-cdn.com/static/image/logo_t.png"><meta itemprop="url" content="https://groups.tapatalk-cdn.com/static/image/logo_t.png"><meta itemprop="width" content="50"><meta itemprop="height" content="50"></div><meta itemprop="url" content="https://tapatalk.com"><meta itemprop="name" content="Tapatalk"></div><meta itemprop="dateModified" content="2021-03-10 12:30:50"><a class="hide" itemprop="mainEntityOfPage" href="./viewtopic.php?f=196&amp;t=193712"><h1 itemprop="name">Will we ever see normal again?</h1></a><i id="p1510189"></i><i id="pfirst"></i><h1 class="hide" itemprop="headline">Will we ever see normal again?</h1><div class="post-profile-block"><dl class="postprofile" id="profile1510189" data-uid="10764548" style=""><dt class="no-profile-rank has-avatar avatar-username"><div class="avatar-username-inner responsive-hide"><div class="avatar-container"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div></div><div style="padding-top:10px;"><span class="thread-user-name" itemprop="author" itemscope="" itemtype="http://schema.org/Person"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" style="color: #008000;" class="username-coloured" itemprop="name">BigSweet</a><div class="thread-user-detail-padding runned_js"><div class="thread-user-detail"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div><div class="thread-user-name" style="max-width:unset; padding-top:10px;"><strong><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" style="color: #008000;" class="username-coloured" itemprop="name">BigSweet</a></strong><div class="popup user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>899</span></span><span class="nowrap"><i class="fa fa-gift"></i><span>16</span></span></div><div style="border-bottom: 1px solid #F0F0f0;width: 95%;margin: auto;"></div><div class="custom_field"><div class="custom_field_key">Joined</div><div class="custom_field_value"><span class="timespan" title="1:34 PM - Aug 30, 2018">Aug 30, 2018</span></div></div>                <div class="custom_field"><div class="custom_field_key">Last active</div><div class="custom_field_value"><span class="timespan" title="11:49 AM - Mar 10, 2021">33 minutes ago</span></div></div></div></div></div></span><div class="user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>899</span></span><span class="nowrap"><i class="fa fa-thumbs-up"></i><span>1,543</span></span><span class="nowrap"><i class="fa fa-gift"></i><span>16</span></span><div class="cl-af"></div></div></div></div><section class="desktop-hide responsive-show"><div class="avatar-container cl-af"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div></div><div class="username postusername"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10764548" style="color: #008000;" class="username-coloured" itemprop="name">BigSweet</a></div><div class="user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>899</span></span><span class="nowrap"><i class="fa fa-thumbs-up"></i><span>1,543</span></span><span class="nowrap"><i class="fa fa-gift"></i><span>16</span></span></div><div class="cl-af"></div></section></dt></dl></div><script>
             var post_data_1510189 = {"POST_AUTHOR":"BigSweet","S_CAN_GIVE_AWARD":0,"U_EDIT":0,"DELETE_VIP_ADS":0,"U_REPORT":0,"S_POST_LOCKED":0,"POSTER_ID":"10764548","S_POST_CONTACT_NAME":"Send private message","S_POST_CONTACT_URL":"https:\/\/vetsbenefits.net\/ucp.php?mode=login&iframe=1","U_DELETE":"","S_POST_HIDDEN":0,"U_HAS_PROFILE_ACTION":1,"MEMBER_IS_VIP_PLUS":"0","POSTER_AVATAR":"<i class=\"avatar-bg\" style=background-image:url(https:\/\/groups.tapatalk-cdn.com\/static\/image\/no_avatar.png) ><\/i>","U_POST_AUTHOR":"https:\/\/vetsbenefits.net\/memberlist.php?mode=viewprofile&amp;u=10764548","HIDE_VIP_PLUS":0,"PROFILE_HAT":0,"POST_AUTHOR_FULL":"<a href=\"https:\/\/vetsbenefits.net\/memberlist.php?mode=viewprofile&amp;u=10764548\" style=\"color: #008000;\" class=\"username-coloured\" itemprop=\"name\">BigSweet<\/a>","S_ONLINE":0,"POSTER_POSTS":"899","POSTER_LIKED":"1,543","POSTER_GIFTED":"16","POSTER_JOINED":"<span class=\"timespan\" title=\"1:34 PM - Aug 30, 2018\">Aug 30, 2018<\/span>","POSTER_LAST_VISIT":"<span class=\"timespan\" title=\"11:49 AM - Mar 10, 2021\">33 minutes ago<\/span>","U_ADD_FRIEND":"https:\/\/vetsbenefits.net\/ucp.php?i=zebra&amp;add=BigSweet","U_IS_FRIEND":0,"U_POST_TTID":"29124088","S_GIVEN_AWARD":0,"S_POST_UNAPPROVED":0,"S_POST_DELETED":0,"U_QUOTE":1,"EDIT_VIP_ADS":0,"S_CAN_LIKE":1,"S_RECOMMEND_LIKE":true,"S_LIKED":0,"S_CAN_DISLIKE":0,"S_DISLIKED":0,"U_PIN_TOPIC":0,"S_CAN_GIVE_VIP":0,"S_GUEST_POSTING":0};
        </script><div id="post_container1510189" class="postbody" style=""><i class="arrow-before"></i><div class="float-arrow-block"><div class="float-arrow-sticky"><i class="arrow-after bg2"></i></div></div><div id="post_content1510189" class="normal-bubble post_content post-bubble-outter-padding bg2"><div id="post_content1510189_award_block"></div><div id="post_content1510189_card_bg_block" class="post-bubble-container "></div><div class="pb1"><ul class="post-buttons runned_js">                      <li class="dropdown-container dropdown-loaded">                            <a href="javascript:void(0);" class="dropdown-trigger button button-icon-only dropdown-toggle">                                <i class="icon fa-fw fa-angle-down" aria-hidden="true"></i>                            </a>                            <div class="dropdown">                                <div class="pointer">                                    <div class="pointer-inner"></div>                                </div>                                <ul class="dropdown-contents dropdown-nonscroll ">    <li>       <a href="0" title="Report this post" class="button-icon-only">           <span>Report this post</span>       </a>    </li>    <li class="profile-contact profile-contact-pm">       <a href="javascript:void(0)" onclick="showPMCompose('https://vetsbenefits.net/ucp.php?mode=login&amp;iframe=1')" title="Send private message">           <span>Send private message</span>       </a>    </li>                                </ul>                            </div>                        </li><li class="responsive-menu hidden dropdown-container"><a href="javascript:void(0);" class="js-responsive-menu-link responsive-menu-link dropdown-toggle"><i class="icon fa-bars fa-fw" aria-hidden="true"></i></a><div class="dropdown"><div class="pointer"><div class="pointer-inner"></div></div><ul class="dropdown-contents"></ul></div></li></ul><p class="author"><i class="icon fa-clock-o"></i><span class="posttime"><span class="timespan" title="2:56 PM - Mar 06, 2021">2:56 PM - 3 days ago</span></span><a class="unread" href="https://vetsbenefits.net/will-we-ever-see-normal-again-t193712.html#p1510189" title="Post"><span>#1</span></a><i class="post-client-type tooltip client-type-100 tooltipstered"></i><time class="hide" itemprop="datePublished" datetime="2021-03-06T19:56">2021-03-06T19:56</time></p></div><div class="pb2  postcontentheight  "><div data-tag="post_body_start" itemprop="articleBody" class="content noskim">There are new more contagious Covid variants popping up all over the world that may or may not be susceptible to the current vaccines and they are headed this way.&nbsp; It is reported that a full 30% of active duty military personnel are refusing to be vaccinated and that the civilian population is pretty much following suit.&nbsp; Many states have begun to prematurely lift current restrictions designed to curb new infections against the advice of the medical experts.&nbsp; The innoculation rates, though improving, are still lagging pitifully behind schedule.&nbsp; Worst of all, after a month or more of greatly declining numbers, there has been a distinct uptick in the number of new daily infections which will invariably lead to more deaths.&nbsp; It begins anew.<br>
<br>
We have been here before already on several occasions.&nbsp; Things begin to look a little rosier, then reality comes back to kick us square in the ass.&nbsp; The general populous tires of following the rules and restrictions put in place for their own good.&nbsp; We simply have to dine out. We simply have to gather with friends and family.&nbsp; We simply can't remain trapped at home.&nbsp; Logic would then follow that we will simply have to die once again in greater numbers.&nbsp; A hard but sad truth none the less.<br>
<br>
We would all love to see things get back to some semblance of normalcy.&nbsp; &nbsp;It will be wonderful when we can once again gather together with those we love without fear.&nbsp; It will be wonderful when we can freely get out and about again.&nbsp; It would be even more wonderful if we had the gumption it takes to sacrifice what need be in order for that day to get here sooner rather than later.&nbsp; If we are not willing to make those sacrifices necessary to put an end to these cycles of reinfection we are likely dooming ourselves to history continually repeating itself and the dreaded "new normal" will continue to be our way of life.<br>
<br>
It will take every one of us doing our part, no matter how insignificant we may think it is, to get this monster finally under control and put behind us.&nbsp; &nbsp;Lord willing, we will succeed.<br>
<br>
&nbsp;<i data-tag="post_body_end" class="hide"></i></div></div><div class="pb3"><div id="postcontent_footer_icon_area_1510189" class="postcontent_footer cl-af" style="min-height:20px">                <div class="postcontent_button" data-quote-postid="1510189" onclick="hs_guest_join(true);gaSendEvent('guest-convert', 'use_feature', 'quote');">
                        <a class="hover" title="Reply with quote">
                        <i class="fa fa-quote-left"></i><span>Quote</span>
                    </a>
                    </div><div class="postcontent_button postcontent_button_like  with-task-bubble ">              <a class="hover" title="Login" onclick="hs_guest_join(true);gaSendEvent('guest-convert', 'use_feature', 'like');">
                                        <i class="fa fa-thumbs-o-up"></i><span>Like</span>
                 </a></div>                    <div class="sr-b dropdown-container dropdown-right postcontent_button dropdown-loaded">
                            <span class="dropdown-trigger dropdown-toggle" onclick="share_gaSendEvent(this, 'click')" title="Share">
                            <a>
                            <i class="fa fa-share-alt"></i><span class="">Share</span>
                        </a>
                        </span><div class="pp-bdr sr-dp dropdown share-index ">        <div class="lbl_1 cl-af">
    <span class="l">Share with:</span>
    <span class="r">
        <i class="sr-ic b fb" onclick="share_gaSendEvent(this, 'facebook')"></i>
        <i class="sr-ic b tw" onclick="share_gaSendEvent(this, 'twitter')"></i>
        <i class="sr-ic b tl" onclick="share_gaSendEvent(this, 'tumble')"></i>
    </span></div>
    <div class="lbl_2">
        <span class="l">Link:</span>
    <span class="r">
        <input type="text" id="cp_1510189" onfocus="this.select();" class="inputbox js-bound" title="Link" readonly="" value="https://vetsbenefits.net/viewtopic.php?p=1510189#p1510189"> <span class="button cp-btn tapatalk-theme-background" data-clipboard-target="#cp_1510189" onclick="share_gaSendEvent(this, 'copy')">Copy link</span>
        </span>
        </div>
        <i class="fa fa-close btn-cls"></i>
</div></div></div><div class="1510189"><div name="1510189-like-count" class="like-count cl-af"><div name="1510189-like-box" class="box" data-post_id="1510189"><div class="icon"><i class="award-icon like"></i></div><span name="1510189-like-preview-users" class="preview-users"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=7342051" class="username" itemprop="name">Bigdmn</a>, <a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10763249" class="username" itemprop="name">Road-house</a>, <a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=7393551" class="username" itemprop="name">Namvet6870</a>, <a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=894747" class="username" itemprop="name">Pistolero</a>, <a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=7194743" class="username" itemprop="name">Ron91</a> plus <span class="tapatalk-theme-color">2</span> additional people liked this post</span></div><div name="1510189-like-user-list-background" class="like-user-list-close like-user-list-background hide"><div class="like-user-list"><div class="sum"><span name="1510189-like-sum-num" class="sum-num">7 people like this post</span><i class="like-user-list-close fa fa-close"></i></div><div class="scroll-container"><div class="scroll-content"><ul></ul><div name="1510189-like-more" class="more">MORE</div></div></div></div></div></div></div></div><div id="recommend_topics_default" class="hide mt10"></div><div id="recommend_topics_1510189" class="hide mt10"></div></div><!-- #post_content .postbody --></div><hr class="divider"></div>

In [ ]:
<div id="p_1349188" itemscope="" itemtype="https://schema.org/UserComments" class="change_color post post-bubble-padding postrow incountiously has-profile  rowtargetads has-profile-block   runned_js"><i id="p1349188"></i><div class="post-profile-block" style="height: 161px;"><dl class="postprofile" id="profile1349188" data-uid="10761745" style=""><dt class="no-profile-rank has-avatar avatar-username"><div class="avatar-username-inner responsive-hide"><div class="avatar-container"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div></div><div style="padding-top:10px;"><span class="thread-user-name"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" style="color: #008000;" class="username-coloured" itemprop="name">MPVET34</a><div class="thread-user-detail-padding runned_js"><div class="thread-user-detail"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div><div class="thread-user-name" style="max-width:unset; padding-top:10px;"><strong><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" style="color: #008000;" class="username-coloured" itemprop="name">MPVET34</a></strong><div class="popup user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>2</span></span></div><div style="border-bottom: 1px solid #F0F0f0;width: 95%;margin: auto;"></div><div class="custom_field"><div class="custom_field_key">Joined</div><div class="custom_field_value"><span class="timespan" title="10:16 AM - Dec 24, 2017">Dec 24, 2017</span></div></div>                <div class="custom_field"><div class="custom_field_key">Last active</div><div class="custom_field_value"><span class="timespan" title="4:50 PM - Feb 01, 2018">Feb 01, 2018</span></div></div></div></div></div></span><div class="user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>2</span></span><div class="cl-af"></div></div></div></div><section class="desktop-hide responsive-show"><div class="avatar-container cl-af"><div class="avatar-frame"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" class="avatar avatar-frame-padding"><i class="avatar-bg" style="background-image:url(https://groups.tapatalk-cdn.com/static/image/no_avatar.png)"></i></a></div></div><div class="username postusername"><a href="https://vetsbenefits.net/memberlist.php?mode=viewprofile&amp;u=10761745" style="color: #008000;" class="username-coloured" itemprop="name">MPVET34</a></div><div class="user-statistics"><span class="nowrap"><i class="fa fa-comment"></i><span>2</span></span></div><div class="cl-af"></div></section></dt></dl></div><script>
             var post_data_1349188 = {"POST_AUTHOR":"MPVET34","S_CAN_GIVE_AWARD":0,"U_EDIT":0,"DELETE_VIP_ADS":0,"U_REPORT":0,"S_POST_LOCKED":0,"POSTER_ID":"10761745","S_POST_CONTACT_NAME":"Send private message","S_POST_CONTACT_URL":"https:\/\/vetsbenefits.net\/ucp.php?mode=login&iframe=1","U_DELETE":"","S_POST_HIDDEN":0,"U_HAS_PROFILE_ACTION":1,"MEMBER_IS_VIP_PLUS":false,"POSTER_AVATAR":"<i class=\"avatar-bg\" style=background-image:url(https:\/\/groups.tapatalk-cdn.com\/static\/image\/no_avatar.png) ><\/i>","U_POST_AUTHOR":"https:\/\/vetsbenefits.net\/memberlist.php?mode=viewprofile&amp;u=10761745","HIDE_VIP_PLUS":0,"PROFILE_HAT":0,"POST_AUTHOR_FULL":"<a href=\"https:\/\/vetsbenefits.net\/memberlist.php?mode=viewprofile&amp;u=10761745\" style=\"color: #008000;\" class=\"username-coloured\" itemprop=\"name\">MPVET34<\/a>","S_ONLINE":0,"POSTER_POSTS":"2","POSTER_LIKED":"0","POSTER_GIFTED":"0","POSTER_JOINED":"<span class=\"timespan\" title=\"10:16 AM - Dec 24, 2017\">Dec 24, 2017<\/span>","POSTER_LAST_VISIT":"<span class=\"timespan\" title=\"4:50 PM - Feb 01, 2018\">Feb 01, 2018<\/span>","U_ADD_FRIEND":"https:\/\/vetsbenefits.net\/ucp.php?i=zebra&amp;add=MPVET34","U_IS_FRIEND":0,"U_POST_TTID":"25369117","S_GIVEN_AWARD":0,"S_POST_UNAPPROVED":0,"S_POST_DELETED":0,"U_QUOTE":1,"EDIT_VIP_ADS":0,"S_CAN_LIKE":1,"S_RECOMMEND_LIKE":false,"S_LIKED":0,"S_CAN_DISLIKE":0,"S_DISLIKED":0,"U_PIN_TOPIC":0,"S_CAN_GIVE_VIP":0,"S_GUEST_POSTING":0};
        </script><div id="post_container1349188" class="postbody" style=""><i class="arrow-before"></i><div class="float-arrow-block" style="height: 11px;"><div class="float-arrow-sticky"><i class="arrow-after bg2"></i></div></div><div id="post_content1349188" class="normal-bubble post_content post-bubble-outter-padding bg2"><div id="post_content1349188_award_block"></div><div id="post_content1349188_card_bg_block" class="post-bubble-container "></div><div class="pb1"><ul class="post-buttons runned_js">                      <li class="dropdown-container dropdown-loaded">                            <a href="javascript:void(0);" class="dropdown-trigger button button-icon-only dropdown-toggle">                                <i class="icon fa-fw fa-angle-down" aria-hidden="true"></i>                            </a>                            <div class="dropdown">                                <div class="pointer">                                    <div class="pointer-inner"></div>                                </div>                                <ul class="dropdown-contents dropdown-nonscroll ">    <li>       <a href="0" title="Report this post" class="button-icon-only">           <span>Report this post</span>       </a>    </li>    <li class="profile-contact profile-contact-pm">       <a href="javascript:void(0)" onclick="showPMCompose('https://vetsbenefits.net/ucp.php?mode=login&amp;iframe=1')" title="Send private message">           <span>Send private message</span>       </a>    </li>                                </ul>                            </div>                        </li><li class="responsive-menu hidden dropdown-container"><a href="javascript:void(0);" class="js-responsive-menu-link responsive-menu-link dropdown-toggle"><i class="icon fa-bars fa-fw" aria-hidden="true"></i></a><div class="dropdown"><div class="pointer"><div class="pointer-inner"></div></div><ul class="dropdown-contents"></ul></div></li></ul><p class="author"><i class="icon fa-clock-o"></i><span class="posttime"><span class="timespan" title="10:20 AM - Dec 24, 2017">Dec 24, 2017</span></span><a class="unread" href="https://vetsbenefits.net/female-vets-read-this-t179780-s80.html#p1349188" title="Post"><span>#87</span></a><time class="hide" itemprop="commentTime" datetime="2017-12-24T15:20">2017-12-24T15:20</time></p></div><div class="pb2  postcontentheight  "><div data-tag="post_body_start" itemprop="commentText" class="content noskim">I would like access please. Thank you.<i data-tag="post_body_end" class="hide"></i></div></div><div class="pb3"><div id="postcontent_footer_icon_area_1349188" class="postcontent_footer cl-af" style="min-height:20px">                <div class="postcontent_button" data-quote-postid="1349188" onclick="hs_guest_join(true);gaSendEvent('guest-convert', 'use_feature', 'quote');">
                        <a class="hover" title="Reply with quote">
                        <i class="fa fa-quote-left"></i><span>Quote</span>
                    </a>
                    </div><div class="postcontent_button postcontent_button_like ">              <a class="hover" title="Login" onclick="hs_guest_join(true);gaSendEvent('guest-convert', 'use_feature', 'like');">
                                        <i class="fa fa-thumbs-o-up"></i><span>Like</span>
                 </a></div>                    <div class="sr-b dropdown-container dropdown-right postcontent_button dropdown-loaded">
                            <span class="dropdown-trigger dropdown-toggle" onclick="share_gaSendEvent(this, 'click')" title="Share">
                            <a>
                            <i class="fa fa-share-alt"></i><span class="">Share</span>
                        </a>
                        </span><div class="pp-bdr sr-dp dropdown share-index ">        <div class="lbl_1 cl-af">
    <span class="l">Share with:</span>
    <span class="r">
        <i class="sr-ic b fb" onclick="share_gaSendEvent(this, 'facebook')"></i>
        <i class="sr-ic b tw" onclick="share_gaSendEvent(this, 'twitter')"></i>
        <i class="sr-ic b tl" onclick="share_gaSendEvent(this, 'tumble')"></i>
    </span></div>
    <div class="lbl_2">
        <span class="l">Link:</span>
    <span class="r">
        <input type="text" id="cp_1349188" onfocus="this.select();" class="inputbox js-bound" title="Link" readonly="" value="https://vetsbenefits.net/viewtopic.php?p=1349188#p1349188"> <span class="button cp-btn tapatalk-theme-background" data-clipboard-target="#cp_1349188" onclick="share_gaSendEvent(this, 'copy')">Copy link</span>
        </span>
        </div>
        <i class="fa fa-close btn-cls"></i>
</div></div></div><div class="1349188"></div></div></div><!-- #post_content .postbody --></div><hr class="divider"></div>

In [1]:
post_dict = {}

In [ ]:
def capture_post_dict(post):
    
    #Get soup info
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')   
    
    tempdic = {}
    soup2.find('h2', class_ = "topic-title").text
    #Get post IDs
    tempdic['topic_title'] = soup2.find('h2', class_ = "topic-title").text
    
    for a in soup.find_all('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
                                  "normal-bubble post_content post-bubble-outter-padding bg2"], id=True): 
    post_id_container.append(a['id'])
    original_post_id_container.append(original_post['id'])

In [138]:
# Global data containers
post_id_container = []
original_post_id_container = []
post_text=[]
post_time = []
post_author=[]

In [ ]:
def capture_post(url):    
    ############
    # New Post #
    ############
    
    #Get soup info
    source_code = requests.get(url+'.html')
    soup = BeautifulSoup(source_code.text, 'html5lib')
    
    #Get post IDs
    original_post = soup.find('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
                                      "normal-bubble post_content post-bubble-outter-padding bg2"], id=True)

    for a in soup.find_all('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
                                      "normal-bubble post_content post-bubble-outter-padding bg2"], id=True): 
        post_id_container.append(a['id'])
        original_post_id_container.append(original_post['id'])
    
    #Get post text
    for element in soup.find_all("div", class_="content noskim"):
        post_text.append(element.text)
    
    #Get Post Time
    for a in soup.find_all('span', class_="timespan", title=True): 
        if a.text: 
            post_time.append(a['title'])
    
    #Get poster name
    for element in soup.find_all('script', text = re.compile("POST_AUTHOR")):

        J = str(element)
        J1 = J.split('{')
        J2 = J1[1].split(',')
        post_author.append(J2[0].split(':'))

    print('finished page 1')
    
#     # Check if next button is present
    next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

#     if next_button:
#         print("next url is " + url + '-s{}.html'.format(40))
#         continue 
#     else:
#         print('no more pages')
#         return None
    
    ###################
    # Loop Thru Pages #
    ###################    

    if next_button:
        s = 40
        counter = 2

        while True:

            source_code = requests.get(url + '-s{}.html'.format(s))
            soup = BeautifulSoup(source_code.text, 'html5lib')

            #Get post IDs
            for a in soup.find_all('div', class_=["normal-bubble post_content post-bubble-outter-padding bg1",
                                              "normal-bubble post_content post-bubble-outter-padding bg2"], id=True): 
                post_id_container.append(a['id'])
                original_post_id_container.append(post_id_container[0])

            #Get post text
            for element in soup.find_all("div", class_="content noskim"):
                post_text.append(element.text)

            #Get Post Time
            for a in soup.find_all('span', class_="timespan", title=True): 
                if a.text: 
                    post_time.append(a['title'])

            #Get poster name
            for element in soup.find_all('script', text = re.compile("POST_AUTHOR")):

                J = str(element)
                J1 = J.split('{')
                J2 = J1[1].split(',')
                post_author.append(J2[0].split(':'))

            next_button = soup.find('i', class_="icon fa-chevron-right fa-fw")

            if next_button:
                s += 40
                print("done page {}".format(counter))
                print("next url is " + url + '-s{}.html'.format(s))
                counter += 1
            else:
                break 

            sleep(randint(5,10))
    else:
        return None

In [144]:
# Zip all files
post_all_zip = zip(post_id_container, post_text, post_time,post_author,original_post_id_container)#,
                  #original_post, original_post_time, original_post_poster_name) 
zipped_post = list(post_all_zip) #Convert zip to list.
df_zipped_post2 = pd.DataFrame(zipped_post, columns=['post_id_container','post_text','post_time','post_author',
                                                  'original_post_id_container'])

In [ ]:
for i in df_discussion_links.url:
    capture_post(i)